In [1]:
import pandas as pd
import re

In [3]:
df = pd.read_csv("../../datasets/WELFake_cleaned.csv")

In [3]:
len(df)

72095

In [4]:
pd.set_option('display.max_colwidth', None)
print(df['text'].iloc[3])

A dozen politically active pastors came here for a private dinner Friday night to hear a conversion story unique in the context of presidential politics: how Louisiana Gov. Bobby Jindal traveled from Hinduism to Protestant Christianity and, ultimately, became what he calls an “evangelical Catholic.”

Over two hours, Jindal, 42, recalled talking with a girl in high school who wanted to “save my soul,” reading the Bible in a closet so his parents would not see him and feeling a stir while watching a movie during his senior year that depicted Jesus on the cross.

“I was struck, and struck hard,” Jindal told the pastors. “This was the Son of God, and He had died for our sins.”

Jindal’s session with the Christian clergy, who lead congregations in the early presidential battleground states of Iowa and South Carolina, was part of a behind-the-scenes effort by the Louisiana governor to find a political base that could help propel him into the top tier of Republican candidates seeking to run f

In [4]:
print(df.sample(6))

       Unnamed: 0                                              title  \
46202       52719  Alienated and Angry, Coal Miners See Donald Tr...   
57094       67023  Williams Sisters Could Be in Step for Semifina...   
36427       40633  Nationalists facing wipe out in Australian sta...   
34478       38288  ’The Legend of Zelda: Breath of the Wild’ - Ni...   
27539       30062  NY attorney general slams Trump's comment on j...   
26249       28572  Matteo Renzi Resigns, Ending Italy’s 63rd Gove...   

                                                    text  label language  \
46202  POWELLTON, W. Va.  —   Deep in the belly of an...      0       en   
57094  If the Williams sisters are to face off in Art...      0       en   
36427  The resurgence of Australian nationalist polit...      0       en   
34478  Check out the latest gameplay from the upcomin...      0       en   
27539  New York’s attorney general, who has filed a l...      0       en   
26249  Matteo Renzi officially stepped 

In [5]:
df['language'].value_counts()

language
en         60299
ru           155
es           141
de           102
unknown       55
fr            35
ar            19
pt             9
sw             8
tr             7
pl             4
it             4
no             4
nl             3
tl             2
el             2
so             1
hr             1
vi             1
ro             1
af             1
zh-cn          1
sq             1
Name: count, dtype: int64

In [31]:
import textwrap

index = 29790                
  # poți schimba cu orice alt rând vrei să vezi
text = df["text"].iloc[index]

wrapped = textwrap.fill(text, width=100)
print(wrapped)

 The U.S. is the great enemy of mankind!  raved Ernesto  Che  Guevara in 1961.  Against those hyenas
there is no option but extermination. We will bring the war to the imperialist enemies  very home,
to his places of work and recreation. The imperialist enemy must feel like a hunted animal wherever
he moves. Thus we ll destroy him! We must keep our hatred against them (the U.S.) alive and fan it
to paroxysms!  -Che Guevara, head of Cuba s  Foreign Liberation Department,  Nov. 17, 1962We have a
president who won t tie Islamic terrorists to Islam, has cut a one-sided deal with Iran, as they
continue to chant  death to America,  and has made an open alliance with a communist country who has
a long history of hating America. The same country Obama is fighting to normalize relations with,
would have committed an unspeakable act of terrorism against us, had our FBI not thwarted their
plans.  Why the rush to embrace so many nations and leaders who have a long history of wanting to
harm us Bar

In [14]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-21 20:29:19.545495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
model = RobertaForSequenceClassification.from_pretrained("saved_models/roberta2_torch_model")
tokenizer = RobertaTokenizer.from_pretrained("saved_models/roberta2_tokenizer")

In [23]:
def evaluate_article(text):
    """
    Preia o știre din DataFrame folosind indexul, o tokenizarează și folosește modelul pentru a o evalua.
    Se presupune că eticheta 0 corespunde știrii false și eticheta 1 știrii reale.
    """
    

    # Tokenizează textul; se utilizează truncation și padding pentru a se asigura că input-ul este adecvat pentru model
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Obține predicția modelului fără calculul gradientului
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits

    # Se alege clasa cu cea mai mare probabilitate
    prediction = torch.argmax(logits, dim=1).item()
    label = "reală" if prediction == 0 else "falsă"
    print(f"Știrea cu indexul {index} este clasificată ca: {label}")

In [ ]:
article_id = 44579     

article = df.iloc[index]


text = article["text"]
evaluate_article(text)

Știrea cu indexul 29790 este clasificată ca: falsă


In [5]:
def remove_city_prefix(text):
    """
    Elimină prefixul ce conține numele orașelor scrise cu majuscule,
    eventual separate prin '/' și urmate de unul sau mai multe caractere dash.
    Exemplu: "CLEVELAND/NEW YORK - " sau "NEW DELHI  —   " vor fi eliminate.
    """
    return re.sub(r'^[A-Z\s/]+[-–—]+\s*', '', text)

texts = [
    "CLEVELAND/NEW YORK - Bracing for a general election fight...",
    "NEW YORK -- Bye bye June rate hike. That was the billboard-si...",
    "NEW DELHI  —   Prime Minister Narendra Modi...",
    "This is an article without prefix."
]

for text in texts:
    print("Original:", text)
    print("Fără prefix:", remove_city_prefix(text))
    print()

Original: CLEVELAND/NEW YORK - Bracing for a general election fight...
Fără prefix: Bracing for a general election fight...

Original: NEW YORK -- Bye bye June rate hike. That was the billboard-si...
Fără prefix: Bye bye June rate hike. That was the billboard-si...

Original: NEW DELHI  —   Prime Minister Narendra Modi...
Fără prefix: Prime Minister Narendra Modi...

Original: This is an article without prefix.
Fără prefix: This is an article without prefix.



In [6]:
def remove_reuters_prefix(text):
    return re.sub(r'^.*?\(Reuters\)\s*-?\s*', '', text)

text_with_reuters = "WASHINGTON DC/Cleveland - (Reuters) This is a sample news article after the prefix."

text_without_reuters = "This is a sample news article without the prefix."

print("Text original cu (Reuters):")
print(text_with_reuters)
print("Text după aplicarea funcției:")
print(remove_reuters_prefix(text_with_reuters))
print("\nText original fără (Reuters):")
print(text_without_reuters)
print("Text după aplicarea funcției:")
print(remove_reuters_prefix(text_without_reuters))

Text original cu (Reuters):
WASHINGTON DC/Cleveland - (Reuters) This is a sample news article after the prefix.
Text după aplicarea funcției:
This is a sample news article after the prefix.

Text original fără (Reuters):
This is a sample news article without the prefix.
Text după aplicarea funcției:
This is a sample news article without the prefix.


In [7]:
df["text"] = df["text"].apply(remove_city_prefix)
df["text"] = df["text"].apply(remove_reuters_prefix)

In [11]:
#drop column Category and clean text
df = df.drop(columns=["clean_text"])

In [13]:
df.to_csv("../../datasets/WELFake.csv", index=False)

In [12]:
df.sample(5)

Unnamed: 0  \
15518       15526   
56863       56895   
39668       39686   
60676       60711   
52270       52298   

                                                                                                                                                          title  \
15518                                                                                                  May weathers new challenges on Brexit plan, more to come   
56863                                                                                      Ted Cruz jeered for refusing to back Trump, exposing Republican rift   
39668                                                                    HILARIOUS! FOX News’ Greg Gutfeld Introduces New Sponsor Of His Show…”Victima” [VIDEO]   
60676  CHER TELLS TWITTER Followers To Open Their Homes To “DREAMERS”…Conservative Twitter User Shares Her Address To Make It Easier For “Dreamers” To Find Her   
52270                                                                             Tony Blair considering return to politics because that’s what happens in 2016   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
filtered_df = df[df['text'].str.contains("(Reuters)", case=False, na=False) & (df['label'] == 0)]

print("Numărul de știri false ce conțin '(Reuters)' în text:", filtered_df.shape[0])

/tmp/ipykernel_214133/720343750.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['text'].str.contains("(Reuters)", case=False, na=False) & (df['label'] == 0)]


Numărul de știri false ce conțin '(Reuters)' în text: 21643


In [13]:
filtered_df.sample(5)

,Unnamed: 0,title,text,label
6926,6926,White House proposes to privatize air traffic ...,WASHINGTON (Reuters) - President Donald Trump ...,0
30753,30753,White House's Cohn says now sees more than 3 p...,WASHINGTON (Reuters) - White House economic ad...,0
34211,34211,"NAFTA envoys lay out proposals, try to block T...",MEXICO CITY (Reuters) - Trade negotiators from...,0
7529,7529,Russian nuclear bombers fly near North Korea i...,MOSCOW (Reuters) - Russian nuclear-capable str...,0
61413,61413,Only one in seven Catalan see dispute with Mad...,MADRID (Reuters) - Just one in seven people fr...,0


In [ ]:
from langdetect import detect

def is_english(text):
    try:
        return detect(text) == 'en'
    except Exception:
        return False

df['is_english'] = df['text'].apply(is_english)

num_english = df['is_english'].sum()
total_stories = len(df)

print("Numărul de știri în engleză:", num_english)
print("Numărul total de știri:", total_stories)

Numărul de știri în engleză: 70703
Numărul total de știri: 72134
